In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Latihan2').getOrCreate()

In [17]:
data=spark.read.csv('contoh2.csv',header=True,inferSchema=True)

In [18]:
data.printSchema()

root
 |-- nama: string (nullable = true)
 |-- gaji: double (nullable = true)
 |-- tunjangan: double (nullable = true)
 |-- tanggungan: double (nullable = true)



In [19]:
data.show()

+-------+------+---------+----------+
|   nama|  gaji|tunjangan|tanggungan|
+-------+------+---------+----------+
|   andi|1000.0|    500.0|       4.0|
|   bary|2000.0|    600.0|       3.0|
|chelsea|1500.0|    400.0|       2.0|
|   dany| 800.0|    200.0|      null|
|   null|  null|   1000.0|      null|
|    edy|1200.0|     null|      null|
|  fajri|1000.0|    100.0|       3.0|
+-------+------+---------+----------+



In [20]:
##Hapus kolom
data.drop('tunjangan').show()

+-------+------+----------+
|   nama|  gaji|tanggungan|
+-------+------+----------+
|   andi|1000.0|       4.0|
|   bary|2000.0|       3.0|
|chelsea|1500.0|       2.0|
|   dany| 800.0|      null|
|   null|  null|      null|
|    edy|1200.0|      null|
|  fajri|1000.0|       3.0|
+-------+------+----------+



In [21]:
##hapus null value
nonNull = data.na.drop()
nonNull.show()

+-------+------+---------+----------+
|   nama|  gaji|tunjangan|tanggungan|
+-------+------+---------+----------+
|   andi|1000.0|    500.0|       4.0|
|   bary|2000.0|    600.0|       3.0|
|chelsea|1500.0|    400.0|       2.0|
|  fajri|1000.0|    100.0|       3.0|
+-------+------+---------+----------+



In [22]:
##how==any
data.na.drop(how="any").show()

+-------+------+---------+----------+
|   nama|  gaji|tunjangan|tanggungan|
+-------+------+---------+----------+
|   andi|1000.0|    500.0|       4.0|
|   bary|2000.0|    600.0|       3.0|
|chelsea|1500.0|    400.0|       2.0|
|  fajri|1000.0|    100.0|       3.0|
+-------+------+---------+----------+



In [23]:
##threshold / minimal 
data.na.drop(how="any",thresh=2).show()

+-------+------+---------+----------+
|   nama|  gaji|tunjangan|tanggungan|
+-------+------+---------+----------+
|   andi|1000.0|    500.0|       4.0|
|   bary|2000.0|    600.0|       3.0|
|chelsea|1500.0|    400.0|       2.0|
|   dany| 800.0|    200.0|      null|
|    edy|1200.0|     null|      null|
|  fajri|1000.0|    100.0|       3.0|
+-------+------+---------+----------+



In [24]:
##Subset
data.na.drop(how="any",subset=['tanggungan']).show()

+-------+------+---------+----------+
|   nama|  gaji|tunjangan|tanggungan|
+-------+------+---------+----------+
|   andi|1000.0|    500.0|       4.0|
|   bary|2000.0|    600.0|       3.0|
|chelsea|1500.0|    400.0|       2.0|
|  fajri|1000.0|    100.0|       3.0|
+-------+------+---------+----------+



In [33]:
### mengisi nilai null (kosong) filling the missing value
data.na.fill('data hilang',['nama','gaji']).show()

+-----------+------+---------+----------+
|       nama|  gaji|tunjangan|tanggungan|
+-----------+------+---------+----------+
|       andi|1000.0|    500.0|       4.0|
|       bary|2000.0|    600.0|       3.0|
|    chelsea|1500.0|    400.0|       2.0|
|       dany| 800.0|    200.0|      null|
|data hilang|  null|   1000.0|      null|
|        edy|1200.0|     null|      null|
|      fajri|1000.0|    100.0|       3.0|
+-----------+------+---------+----------+



In [34]:
#Menambahkan imputer
from pyspark.ml.feature import Imputer

imputer = Imputer(
    inputCols=['gaji','tunjangan','tanggungan'],
    outputCols=["{}_imputed".format(c) for c in ['gaji','tunjangan','tanggungan']]
    ).setStrategy("median")

In [35]:
imputer.fit(data).transform(data).show()
## jadi imputer itu mengisis nilai null dengan value median diantara nilai kolommnya,
##nanti akan terbuat kolom baru untuk mengisi nilai null

+-------+------+---------+----------+------------+-----------------+------------------+
|   nama|  gaji|tunjangan|tanggungan|gaji_imputed|tunjangan_imputed|tanggungan_imputed|
+-------+------+---------+----------+------------+-----------------+------------------+
|   andi|1000.0|    500.0|       4.0|      1000.0|            500.0|               4.0|
|   bary|2000.0|    600.0|       3.0|      2000.0|            600.0|               3.0|
|chelsea|1500.0|    400.0|       2.0|      1500.0|            400.0|               2.0|
|   dany| 800.0|    200.0|      null|       800.0|            200.0|               3.0|
|   null|  null|   1000.0|      null|      1000.0|           1000.0|               3.0|
|    edy|1200.0|     null|      null|      1200.0|            400.0|               3.0|
|  fajri|1000.0|    100.0|       3.0|      1000.0|            100.0|               3.0|
+-------+------+---------+----------+------------+-----------------+------------------+

